In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
from src.prepare_dataset.prepare_dataset import get_data_paths_and_labels_from_machine

2024-07-09 12:42:45.324688: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-09 12:42:45.347915: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 12:42:45.347934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 12:42:45.348550: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 12:42:45.352337: I tensorflow/core/platform/cpu_feature_guar

Initializeing prepare_dataset...


In [3]:
data_paths, labels = get_data_paths_and_labels_from_machine("fan")

base_dir is  /mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data
['data', 'DCASE', '2020', 'dev', 'fan', 'train']
['data', 'DCASE', '2020', 'dev', 'fan', 'test']
['data', 'DCASE', '2020', 'eval', 'fan', 'test']
['data', 'DCASE', '2020', 'add', 'fan', 'train']
['data', 'DCASE', '2021', 'dev', 'fan', 'train']
['data', 'DCASE', '2021', 'dev', 'fan', 'test']
['data', 'DCASE', '2021', 'eval', 'fan', 'test']
['data', 'DCASE', '2021', 'add', 'fan', 'train']
['data', 'DCASE', '2022', 'dev', 'fan', 'train']
['data', 'DCASE', '2022', 'dev', 'fan', 'test']
['data', 'DCASE', '2022', 'eval', 'fan', 'test']
['data', 'DCASE', '2022', 'add', 'fan', 'train']
['data', 'DCASE', '2023', 'dev', 'fan', 'train']
['data', 'DCASE', '2023', 'dev', 'fan', 'test']
['data', 'DCASE', '2024', 'dev', 'fan', 'train']
['data', 'DCASE', '2024', 'dev', 'fan', 'test']
['data', 'MIMII', 'data_-6_db', 'fan', 'id_00', 'normal']
['data', 'MIMII', 'data_-6_db', 'fan', 'id_00', 'abnormal']
['data', 'MIMII', 'data_-

In [4]:
normal_sample_path = data_paths["MIMII"]["data_6_db"]["normal"][:100]
normal_sample_label = [ 1 for _ in range(100) ]
abnormal_sample_path = data_paths["MIMII"]["data_6_db"]["abnormal"][:100]
abnormal_sample_label = [ -1 for _ in range(100) ]

In [5]:
len(normal_sample_path), len(abnormal_sample_path), len(normal_sample_label), len(abnormal_sample_label)

(100, 100, 100, 100)

In [6]:
sample_path_tmp = normal_sample_path[0], normal_sample_path[1]
print(sample_path_tmp)

('/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_6_db/fan/id_00/normal/00000305.wav', '/mnt/d/silofox/paad/anomaly-example/exploring-AAD/notebooks/data/MIMII/data_6_db/fan/id_00/normal/00000611.wav')


In [7]:
from src.preprocess.submodule.file_to_vector import file_to_vector_mel
from src.preprocess.submodule.normalize import min_max_normalization
import tensorflow as tf
import numpy as np

sample_lms = np.zeros((2, 64, 313))
sample_lms[0] = file_to_vector_mel(sample_path_tmp[0])
sample_lms[1] = file_to_vector_mel(sample_path_tmp[1])

min1 = np.min(sample_lms[0])
max1 = np.max(sample_lms[0])

min2 = np.min(sample_lms[1])
max2 = np.max(sample_lms[1])

sample_lms[0] = min_max_normalization(sample_lms[0], min1, max1)
sample_lms[1] = min_max_normalization(sample_lms[1], min2, max2)

sample_lms = tf.cast(sample_lms, tf.float32)

2024-07-09 12:42:47.359789: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-09 12:42:47.388064: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-09 12:42:47.395622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
sample_lms[0].shape

TensorShape([64, 313])

In [9]:
from src.models.submodule.autoencoder import AutoEncoder

In [10]:
autoencoder = AutoEncoder(sample_lms[0].shape)
# autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')
# autoencoder.build(input_shape = sample_lms.shape)

In [11]:
sample_tensor = autoencoder(sample_lms)

In [12]:
sample_tensor.shape

TensorShape([2, 64, 313])

In [13]:
autoencoder.summary()

Model: "auto_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (2, 8)                    1289016   
                                                                 
 sequential_1 (Sequential)   (2, 64, 313)              1309040   
                                                                 
Total params: 2598056 (9.91 MB)
Trainable params: 2598056 (9.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (2, 20032)                0         
                                                                 
 dense (Dense)               (2, 64)                   1282112   
                                                                 
 dense_1 (Dense)             (2, 64)                   4160      
                                                                 
 dense_2 (Dense)             (2, 32)                   2080      
                                                                 
 dense_3 (Dense)             (2, 16)                   528       
                                                                 
 dense_4 (Dense)             (2, 8)                    136       
                                                                 
Total params: 1289016 (4.92 MB)
Trainable params: 128901

In [15]:
autoencoder.decoder.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (2, 16)                   144       
                                                                 
 dense_6 (Dense)             (2, 32)                   544       
                                                                 
 dense_7 (Dense)             (2, 64)                   2112      
                                                                 
 dense_8 (Dense)             (2, 64)                   4160      
                                                                 
 dense_9 (Dense)             (2, 20032)                1302080   
                                                                 
 reshape (Reshape)           (2, 64, 313)              0         
                                                                 
Total params: 1309040 (4.99 MB)
Trainable params: 1309

In [16]:
autoencoder.load_weights("tmp/autoencoder_weights.h5")

In [17]:
autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (2, 20032)                0         
                                                                 
 dense (Dense)               (2, 64)                   1282112   
                                                                 
 dense_1 (Dense)             (2, 64)                   4160      
                                                                 
 dense_2 (Dense)             (2, 32)                   2080      
                                                                 
 dense_3 (Dense)             (2, 16)                   528       
                                                                 
 dense_4 (Dense)             (2, 8)                    136       
                                                                 
Total params: 1289016 (4.92 MB)
Trainable params: 128901

In [18]:
autoencoder.predict(sample_lms)

1/1 [==============================] - 0s 119ms/step


array([[[0.33391067, 0.2849533 , 0.27742293, ..., 0.28879005,
         0.27038875, 0.29557618],
        [0.34278715, 0.30116922, 0.29594615, ..., 0.29992494,
         0.3010144 , 0.30678016],
        [0.32524252, 0.28282917, 0.28246447, ..., 0.2707874 ,
         0.27969033, 0.29376906],
        ...,
        [0.78100306, 0.74375343, 0.7407391 , ..., 0.74329287,
         0.75462544, 0.7432747 ],
        [0.77236545, 0.73721397, 0.7421381 , ..., 0.7535662 ,
         0.7321148 , 0.7503984 ],
        [0.77464116, 0.74750555, 0.77468854, ..., 0.7484343 ,
         0.7593792 , 0.7526678 ]],

       [[0.31592527, 0.2704712 , 0.26659092, ..., 0.27338558,
         0.2646732 , 0.2871846 ],
        [0.32545996, 0.28610334, 0.28069085, ..., 0.2863679 ,
         0.29291442, 0.291672  ],
        [0.3132167 , 0.27277103, 0.27019852, ..., 0.2649137 ,
         0.26880142, 0.2861079 ],
        ...,
        [0.7445925 , 0.70952773, 0.71154916, ..., 0.7062173 ,
         0.71868396, 0.7112382 ],
        [0.7

In [19]:
autoencoder.encoder.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 20032)             0         
                                                                 
 dense (Dense)               (None, 64)                1282112   
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
Total params: 1289016 (4.92 MB)
Trainable params: 128901

In [20]:
autoencoder.predict(sample_lms)

1/1 [==============================] - 0s 16ms/step


array([[[0.33391067, 0.2849533 , 0.27742293, ..., 0.28879005,
         0.27038875, 0.29557618],
        [0.34278715, 0.30116922, 0.29594615, ..., 0.29992494,
         0.3010144 , 0.30678016],
        [0.32524252, 0.28282917, 0.28246447, ..., 0.2707874 ,
         0.27969033, 0.29376906],
        ...,
        [0.78100306, 0.74375343, 0.7407391 , ..., 0.74329287,
         0.75462544, 0.7432747 ],
        [0.77236545, 0.73721397, 0.7421381 , ..., 0.7535662 ,
         0.7321148 , 0.7503984 ],
        [0.77464116, 0.74750555, 0.77468854, ..., 0.7484343 ,
         0.7593792 , 0.7526678 ]],

       [[0.31592527, 0.2704712 , 0.26659092, ..., 0.27338558,
         0.2646732 , 0.2871846 ],
        [0.32545996, 0.28610334, 0.28069085, ..., 0.2863679 ,
         0.29291442, 0.291672  ],
        [0.3132167 , 0.27277103, 0.27019852, ..., 0.2649137 ,
         0.26880142, 0.2861079 ],
        ...,
        [0.7445925 , 0.70952773, 0.71154916, ..., 0.7062173 ,
         0.71868396, 0.7112382 ],
        [0.7

In [21]:
another_sample = np.zeros((3, 64, 313))
another_sample[0] = file_to_vector_mel(normal_sample_path[10])


min3 = np.min(another_sample[0])
max3 = np.max(another_sample[0])

another_sample[0] = min_max_normalization(another_sample[0], min3, max3)
another_sample[0] = tf.cast(another_sample[0], tf.float32)

another_sample[1] = sample_lms[0]
another_sample[2] = sample_lms[1]

In [22]:
autoencoder.predict(another_sample)

1/1 [==============================] - 0s 31ms/step


array([[[0.3100082 , 0.26483163, 0.25779346, ..., 0.26816267,
         0.25186238, 0.27521077],
        [0.3188396 , 0.2803217 , 0.27540582, ..., 0.2784964 ,
         0.27999946, 0.2847881 ],
        [0.3023342 , 0.26292983, 0.26262027, ..., 0.2518498 ,
         0.26015   , 0.27299914],
        ...,
        [0.724214  , 0.68994844, 0.687334  , ..., 0.68933135,
         0.70026445, 0.689506  ],
        [0.716443  , 0.6836467 , 0.6879558 , ..., 0.69910777,
         0.67947954, 0.69618726],
        [0.71862316, 0.693435  , 0.71867245, ..., 0.69425243,
         0.70427346, 0.69826007]],

       [[0.33391067, 0.2849533 , 0.27742293, ..., 0.28879005,
         0.27038875, 0.29557618],
        [0.34278715, 0.30116922, 0.29594615, ..., 0.29992494,
         0.3010144 , 0.30678016],
        [0.32524252, 0.28282917, 0.28246447, ..., 0.2707874 ,
         0.27969033, 0.29376906],
        ...,
        [0.78100306, 0.74375343, 0.7407391 , ..., 0.74329287,
         0.75462544, 0.7432747 ],
        [0.7

In [23]:
result = autoencoder.predict(another_sample)

1/1 [==============================] - 0s 17ms/step


In [24]:
result.shape

(3, 64, 313)

In [25]:
another_sample = np.zeros((1, 64, 313))
another_sample[0] = sample_lms[0]

In [26]:
result = autoencoder.predict(another_sample)

1/1 [==============================] - 0s 19ms/step


In [27]:
result.shape

(1, 64, 313)

In [28]:
from src.preprocess.submodule.file_to_vector import file_to_vector_mel
from src.preprocess.submodule.vector_to_numpy_arr import vector_to_numpy_arr
from src.preprocess.submodule.normalize import min_max_normalization


In [29]:
normal_sample_vec = [ file_to_vector_mel(path_tmp) for path_tmp in normal_sample_path ]
normal_sample_np = vector_to_numpy_arr(normal_sample_vec)
normal_sample = [ min_max_normalization(sample) for sample in normal_sample_np ]

100%|██████████| 100/100 [00:00<00:00, 13893.48it/s]


In [30]:
normal_sample_np.shape

(100, 64, 313)

In [31]:
result = autoencoder.predict(normal_sample_np)

4/4 [==============================] - 0s 2ms/step


In [32]:
result.shape

(100, 64, 313)

: 